# Trabalho Prático Final 

Sistema de recomendação de games

In [166]:
#instalando a lib surprise
!pip install surprise

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [167]:
import pandas as pd


games = pd.read_csv('games.csv')
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
games.columns = ['id_game','titulo','genero']


#Buscando games assistidos e não assistidos de um usuário

In [168]:
games_jogados = notas.query('id_usuario == 1')
games_nao_jogados = games.set_index('id_game').drop(games_jogados.set_index('id_game').index,errors='ignore')
games_nao_jogados.head(5)

,titulo,genero
id_game,,
5,Minecraft,Sandbox
12,Halo: Combat Evolved,Shooters (FPS and TPS)
13,The Last of Us,Survival and horror
14,GoldenEye 007,Shooters (FPS and TPS)
19,Grand Theft Auto: San Andreas,Sandbox


In [169]:
#conferindo os games não jogados
#ele compara o id do usuário e o id do game e se não retornar nada, é porque está correto.
games_jogados = notas.query('id_usuario == 584 and id_game == 1237')
games_jogados

,id_usuario,id_game,nota_usuario_game,dia_avaliacao


#Criando, treinando, salvando e carregando modelo

In [170]:
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split 
import pickle

In [171]:
#reader = escala das notas, a estrutura da linha de entrada de dados -> id_u id_i avaliacao
reader = Reader(line_format='user item rating',rating_scale=(0.0,5.0))

In [172]:
#preparando os dados para a lib surprise, que deve ter somente as colunas id do usuario, id do game e a nota
dados_surprise = notas[['id_usuario','id_game','nota_usuario_game']]
dados_preparados_treinamento = surprise.Dataset.load_from_df(dados_surprise,reader)

In [173]:
#temos os dados preparados em formato objeto surprise
#criando o modelo SVD
algo = surprise.SVDpp()

In [174]:
trainset, testset = train_test_split(dados_preparados_treinamento)

In [175]:
#para realizar o treinamento utiliza-se o algo.fit()
algo.fit(trainset)

In [176]:
pickle.dump(algo,open('treinado.pkl','wb'))

In [177]:
print('Predições dos games não jogados do usuário 1')
dic=[]
#vamos pegar todos os games não jogados, calcular a predição e salvar no dicionario
games_nao_jogados = games_nao_jogados.reset_index()
for index, row in games_nao_jogados.iterrows():
  dic.append(algo.predict(1,row['id_game']))
  

Predições dos games não jogados do usuário 1


In [178]:
predicoes = pd.DataFrame.from_dict(dic)

In [179]:
predicoes.columns = ['id_usuario','id_game','ref','predicao','destalhes']

In [180]:
recomendacoes = pd.merge(predicoes,games,how='inner',on='id_game').sort_values('predicao',ascending=False)[['id_game','titulo','predicao']]

In [181]:
#recomendacoes.head(50)

#Modulando funções para buscar games jogados e não jogados pelo id_game

In [182]:
def busca_games_jogados(id_usuario):
  print('Games jogados do usuário %d' % id_usuario)
  games_jogados = notas.query('id_usuario == %d' % id_usuario)
  return games_jogados

In [183]:
def busca_games_nao_jogados(id_usuario):
  games_jogados = notas.query('id_usuario == %d' % id_usuario)
  id_games_jogados = games_jogados.set_index('id_game').index
  games_nao_jogados = games.set_index('id_game').drop(id_games_jogados,errors='ignore').reset_index()
  return games_nao_jogados


In [184]:
def verifica_predicoes_games_nao_jogados(id_usuario):
  print('Predicoes do usuário %d' % id_usuario)
  dic=[]
  #vamos pegar todos os games não jogados, calcular a predição e salvar no dicionario
  games_predicoes = busca_games_nao_jogados(id_usuario)
  for index, row in games_predicoes.iterrows():
    dic.append(algo.predict(id_usuario,row['id_game']))
  return pd.DataFrame.from_dict(dic)

#Realizando predições

In [185]:
def apresenta_recomendacoes(id_usuario,numero_de_recomendacoes=10):
  games_jogados = notas.query("id_usuario == %d " % id_usuario)
  if games_jogados.empty:
    print("****O usuário com id %d ainda não avaliou nenhum game****" % id_usuario)
    print("****Serão recomendados os games mais populares****")
    total_de_votos = notas['id_game'].value_counts()
    games_backup = games.copy()
    games_backup['total_de_votos'] = total_de_votos
    notas_medias = notas.groupby('id_game').mean()['nota_usuario_game']
    games_backup['notas_medias'] = notas_medias
    return games_backup.sort_values('total_de_votos',ascending=False).head(numero_de_recomendacoes)

  predicoes = verifica_predicoes_games_nao_jogados(id_usuario)
  predicoes.columns = ['id_usuario','id_game','ref','predicao','destalhes']
  recomendacoes = pd.merge(predicoes,games,how='inner',on='id_game').sort_values('predicao',ascending=False)[['id_game','titulo','predicao']]
  return recomendacoes.head(numero_de_recomendacoes)

In [186]:
apresenta_recomendacoes(950,5)

****O usuário com id 950 ainda não avaliou nenhum game****
****Serão recomendados os games mais populares****


,id_game,titulo,genero,total_de_votos,notas_medias
34,35,Dark Souls,Role-playing,12.0,4.666667
12,13,The Last of Us,Survival and horror,12.0,4.250000
5,6,Grand Theft Auto V,Sandbox,11.0,4.454545
23,24,Final Fantasy VII,Role-playing,10.0,4.600000
4,5,Minecraft,Sandbox,10.0,4.100000


In [187]:
apresenta_recomendacoes(584)

Predicoes do usuário 584


,id_game,titulo,predicao
21,22,Super Mario Bros. 3,4.733756
47,48,The Sims,4.712695
33,34,Chrono Trigger,4.700881
16,17,Street Fighter II,4.696003
19,20,Super Mario Bros.,4.661912
8,9,Red Dead Redemption,4.652337
14,15,The Legend of Zelda: Breath of the Wild,4.646798
0,1,The Legend of Zelda: Ocarina of Time,4.622404
5,6,Grand Theft Auto V,4.621152
12,13,The Last of Us,4.589717


#Inserindo dados no arquivo CSV

In [188]:
#CSV
import csv

In [189]:
with open('ratings.csv','a') as arquivo_csv:
  escrever = csv.writer(arquivo_csv,delimiter=',',lineterminator='\n')
  escrever.writerow([584,1223,5.0])

In [190]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']

In [191]:
notas.tail()

,id_usuario,id_game,nota_usuario_game,dia_avaliacao
198,4,4,5.0,964983762.0
199,584,1223,5.0,NaN
200,584,1223,5.0,NaN
201,584,1223,5.0,NaN
202,584,1223,5.0,NaN


In [192]:
notas.tail()

,id_usuario,id_game,nota_usuario_game,dia_avaliacao
198,4,4,5.0,964983762.0
199,584,1223,5.0,NaN
200,584,1223,5.0,NaN
201,584,1223,5.0,NaN
202,584,1223,5.0,NaN


#Salvar predições em arquivo xml

In [193]:
import xml.etree.cElementTree as ET

In [194]:
def grava_arquivo_xml(dados,id_usuario):
    root = ET.Element("Catalogo")
    for index, row in dados.iterrows():
        doc = ET.SubElement(root, "Game")
        ET.SubElement(doc, "Id_game").text = str(row['id_game'])
        ET.SubElement(doc, "Titulo").text = row['titulo']
    tree = ET.ElementTree(root)
    tree.write("games_us_"+str(id_usuario)+".xml")


In [195]:
grava_arquivo_xml(apresenta_recomendacoes(145,5),145)

****O usuário com id 145 ainda não avaliou nenhum game****
****Serão recomendados os games mais populares****


#Função para realizar treinamento

In [196]:
#essa função vai criar o modelo e treinamento com os dados.
#escala menor é a nota mais baixa do conjunto de dados. -> 0.0
#escala maior é a nota mais alta do conjunto de dados. -> 0.5
def realiza_treinamento(dados, nome_modelo,escala_menor, escala_maior):
  if(nome_modelo == 'SVD') or (nome_modelo == 'svd'):
    print('Modelo escolhido foi o SVD')
    algo = surprise.SVD()
    print('Modelo gerado com sucesso')
  elif(nome_modelo == 'SVDpp') or (nome_modelo == 'svdpp') or (nome_modelo == 'SVDPP'):
    print('Modelo escolhido foi o SVDpp')
    algo = surprise.SVDpp()
    print('Modelo gerado com sucesso')

  reader = Reader(line_format='user item rating',rating_scale=(escala_menor,escala_maior))
  dados_surprise = dados[['id_usuario','id_game','nota_usuario_game']]
  dados_preparados_treinamento = surprise.Dataset.load_from_df(dados_surprise,reader)
  trainset, testset = train_test_split(dados_preparados_treinamento)
  algo.fit(trainset)
  print("Modelo "+nome_modelo+" treinado com sucesso")
  return algo


In [197]:
algo = realiza_treinamento(notas,'SVD',0.0,5.0)

Modelo escolhido foi o SVD
Modelo gerado com sucesso
Modelo SVD treinado com sucesso


#Criando menu para inserir avaliações, recarregar treinamento, apresentar recomendações

In [198]:
#função para salvar o modelo treinado
def salva_modelo(algo):
  print(algo)
  pickle.dump(algo,open('treinado.pkl','wb'))
  print("O comando dump foi executado sem erro")

In [199]:
def carrega_modelo():
  with open('treinado.pkl','rb') as modelo:
    algo = pickle.load(modelo)
  return algo

In [200]:
def verifica_predicoes_games_nao_jogados(id_usuario,algo):
  print('Predicoes do usuário %d' % id_usuario)
  dic=[]
  #vamos pegar todos os games não jogados, calcular a predição e salvar no dicionario
  games_predicoes = busca_games_nao_jogados(id_usuario)
  for index, row in games_predicoes.iterrows():
    dic.append(algo.predict(id_usuario,row['id_game']))
  return pd.DataFrame.from_dict(dic)

In [201]:
def apresenta_recomendacoes(id_usuario,notas,algo,games,numero_de_recomendacoes=10):
  games_assistidos = notas.query("id_usuario == %d " % id_usuario)
  if games_assistidos.empty:
    print("****O usuário com id %d ainda não avaliou nenhum game****" % id_usuario)
    print("****Serão recomendados os games mais populares****")
    total_de_votos = notas['id_game'].value_counts()
    games_backup = games.copy()
    games_backup['total_de_votos'] = total_de_votos
    notas_medias = notas.groupby('id_game').mean()['nota_usuario_game']
    games_backup['notas_medias'] = notas_medias
    return games_backup.sort_values('total_de_votos',ascending=False).head(numero_de_recomendacoes)

  predicoes = verifica_predicoes_games_nao_jogados(id_usuario,algo)
  predicoes.columns = ['id_usuario','id_game','ref','predicao','destalhes']
  recomendacoes = pd.merge(predicoes,games,how='inner',on='id_game').sort_values('predicao',ascending=False)[['id_game','titulo','predicao']]
  return recomendacoes.head(numero_de_recomendacoes)

In [202]:
def menu():
  continuar = 1
  while(continuar):
    continuar = int(input("0. Sair\n"+
                          "1. Inserir novas avaliações\n"+
                          "2. Verificar id de um novo usuário\n"+
                          "3. Realiza Treinamento\n"+
                          "4. Recomenda Games\n"
                          )
                    )
    if(continuar == 1):
      usuario = input("Digite o id do usuário: ")
      game = input("Digite o id do game: ")
      nota = input("Digite a nota do game: ")

      with open('ratings.csv','a') as arquivo_csv:
        escrever = csv.writer(arquivo_csv,delimiter=',',lineterminator='\n')
        escrever.writerow([usuario,game,nota])

    if(continuar == 2):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
      novo_usuario = notas['id_usuario'].max()+1
      print("O próximo ID de usuário livre eh: %d" % novo_usuario)

    if(continuar == 3):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
      modelo = input("Digite o nome do modelo: ")
      escala_menor = float(input("Digite o valor da nota mais baixa: "))
      escala_maior = float(input("Digite o valor da nota mais alta: "))
      algo = realiza_treinamento(notas,modelo,escala_menor,escala_maior)
      salva_modelo(algo)

    if(continuar == 4):
      notas = pd.read_csv('ratings.csv')
      notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
      algo = carrega_modelo()
      id_usuario = int(input("Recomendar games para: "))
      numero_de_recomendacoes = int(input("Digite o número de games a recomendar: "))
      recomendacoes = apresenta_recomendacoes(id_usuario,notas,algo,games,numero_de_recomendacoes)
      grava_arquivo_xml(recomendacoes,id_usuario)
      print(recomendacoes[['id_game','titulo']])
      

In [203]:
menu()

Predicoes do usuário 1
    id_game            titulo
18       46  Final Fantasy VI
2        13    The Last of Us


In [204]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['id_usuario','id_game','nota_usuario_game','dia_avaliacao']
notas.tail(5)

,id_usuario,id_game,nota_usuario_game,dia_avaliacao
198,4,4,5.0,964983762.0
199,584,1223,5.0,NaN
200,584,1223,5.0,NaN
201,584,1223,5.0,NaN
202,584,1223,5.0,NaN


#Verificar recomendações após novas avaliações

#Base de dados escolhida

https://github.com/andredarcie/best-games-of-all-time-data-based/blob/master/dataset/generated_game_list.csv